In [ ]:
import argparse
import numpy as np
import torch
from torchvision import datasets, transforms

In [ ]:
parser = argparse.ArgumentParser(
    description='Too simple! Sometimes naive!'
)
parser.add_argument(
    '-s', '--size',
    type=int,
    default=32,
    help='image size (n x n)'
)
parser.add_argument(
    '-m', '--magnitude',
    type=int,
    default=10,
    help='randAugment magnitude'
)
parser.add_argument(
    '-n', '--numblock',
    type=int,
    default=3,
    help='numbers of blocks each layer'
)
parser.add_argument(
    '-w', '--workers',
    type=int,
    default=2,
    help='num_workers'
)
parser.add_argument(
    '-b', '--begin',
    type=int,
    default=512,
    help='begin batch size'
)
parser.add_argument(
    '-e', '--end',
    type=int,
    default=512,
    help='end batch size'
)
parser.add_argument(
    '-g', '--gap',
    type=int,
    default=1,
    help='batch size gap (step size)'
)
# parser.parse_args() is used in .py files
# parser.parse_args('') is used in .ipynb files
args = parser.parse_args()

In [ ]:
# Optimization Setting
AUTOCAST_FLAG = False
COMPILE_FLAG = False

# Data && Model Setting
## Data Resize Shape
size = args.size #16, '32'
## RandAugment
num_ops = 2
magnitude = args.magnitude #5, '10'
## Model
n = args.numblock
num_classes = 100
p = 0 #0.1, '0.2'
## DataLoader
num_workers = args.workers
pin_memory = False
drop_last = True
persistent_workers = False

# Training Setting
#epochs = 1
## begin && end batch size
begin_size = args.begin
end_size = args.end + 1
step_size = args.gap
## loss function
lr = 1e-1
momentum = 0.9
weight_decay = 1e-4

In [ ]:
root = '~/.pytorch/datasets/'
device = torch.device(
    f'cuda:{torch.cuda.device_count() - 1}' if torch.cuda.is_available() else 'cpu'
)
if device.type == 'cuda':
    torch.cuda.set_device(device)
    torch.backends.cudnn.benchmark = True
print(f'Device: {device}, Type: {device.type}')

In [ ]:
# Models
## Lazy Model
class LazyBasicBlock(torch.nn.Module):
    def __init__(self, planes: int, down: bool = False) -> None:
        super().__init__()
        self.down = down
        self.conv1 = (
            torch.nn.LazyConv2d(planes, 3, stride=2, padding=1, bias=False) if down
            else torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        )
        self.bn1 = torch.nn.LazyBatchNorm2d()
        self.conv2 = torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        self.bn2 = torch.nn.LazyBatchNorm2d()
        self.relu = torch.nn.ReLU(inplace=True)
        if self.down:
            self.downsample = torch.nn.Sequential(
                torch.nn.LazyConv2d(planes, 1, stride=2, bias=False),
                torch.nn.LazyBatchNorm2d()
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.down:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class Lazy_CIFAR_ResNet(torch.nn.Module):
    def __init__(self, n: int = 3, num_classes: int = 100, p: float = 0.2) -> None:
        super().__init__()
        self.n = n
        self.planes = 16
        def consruct_layers(self, down: bool = False):
            layers = []
            for i in range(self.n):
                if i == 0 and down == True:
                    self.planes *= 2
                    layers.append(LazyBasicBlock(self.planes, down=True))
                else:
                    layers.append(LazyBasicBlock(self.planes))
            return torch.nn.Sequential(*layers)
        self.stem = torch.nn.Sequential(
            torch.nn.LazyConv2d(self.planes, kernel_size=3, padding='same', bias=False),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.ReLU(inplace=True)
        )
        self.layer1 = consruct_layers(self)
        self.layer2 = consruct_layers(self, down=True)
        self.layer3 = consruct_layers(self, down=True)
        self.classifier = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(1),
            torch.nn.Flatten(),
            torch.nn.Dropout(p=p, inplace=True),
            torch.nn.LazyLinear(num_classes)
        )
    def adjust_dropout(self, p: float = 0.2):
        self.classifier[2] = torch.nn.Dropout(p=p, inplace=True)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.classifier(x)
        return x

In [ ]:
def train_step(model, dataset, dataloader, criterion, optimizer, scaler, AUTOCAST_FLAG=False):
    record_loss, record_acc = 0, 0
    model.train()
    for i, data in enumerate(dataloader):
        # load data
        inputs = data[0].to(device, non_blocking=True)
        labels = data[1].to(device, non_blocking=True)
        # compute
        optimizer.zero_grad()
        with torch.autocast(device.type, enabled=AUTOCAST_FLAG):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # record
        predict_labels = torch.max(outputs, dim=1).indices
        record_loss += loss.item()
        record_acc += torch.sum(labels==predict_labels).item()
    record_loss /= len(dataloader)
    record_acc /= len(dataset)
    return record_loss, record_acc

def eval_step(model, dataset, dataloader, criterion, AUTOCAST_FLAG=False):
    record_loss, record_acc = 0, 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            # load data
            inputs = data[0].to(device, non_blocking=True)
            labels = data[1].to(device, non_blocking=True)
            # compute
            with torch.autocast(device.type, enabled=AUTOCAST_FLAG):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            # record
            predict_labels = torch.max(outputs, dim=1).indices
            record_loss += loss.item()
            record_acc += torch.sum(labels==predict_labels).item()
    record_loss /= len(dataloader)
    record_acc /= len(dataset)
    return record_loss, record_acc

In [ ]:
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

In [ ]:
normalize = transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
transform = {
    'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4, padding_mode='constant'),
        transforms.Resize(size=size),
        transforms.RandomHorizontalFlip(),
        #transforms.RandAugment(num_ops=num_ops, magnitude=magnitude),
        transforms.ToTensor(),
        normalize,
    ]),
    'eval': transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        normalize,
    ])
}
dataset = {
    'train': datasets.CIFAR100(
        root=root, train=True, download=True, transform=transform['train']
    ),
    'test': datasets.CIFAR100(
        root=root, train=False, download=True, transform=transform['eval']
    )
}

In [ ]:
# n = [3, 5, 7, 9, 18]
# layers = 6n + 2
# cifar_resnet = [20, 32, 44, 56, 110]
model = Lazy_CIFAR_ResNet(n=n, num_classes=num_classes, p=p).to(device)

In [ ]:
scaler = torch.cuda.amp.GradScaler(
    enabled=True if device.type=='cuda' and AUTOCAST_FLAG else False
)
# compile_mode: 'default', 'reduce-overhead', 'max-autotune'
model = torch.compile(model, mode='default', fullgraph=True, disable=not COMPILE_FLAG)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay
)

In [ ]:
time_record = []

In [ ]:
for batch_size in range(begin_size, end_size, step_size):
    # dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset['train'],
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers
    )
    # warm up
    for i, data in enumerate(dataloader):
        # load data
        inputs = data[0].to(device, non_blocking=True)
        labels = data[1].to(device, non_blocking=True)
        # compute
        optimizer.zero_grad()
        with torch.autocast(device.type, enabled=AUTOCAST_FLAG):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        break
    # train
    result, time_cost = timed(
        lambda: train_step(model, dataset['train'], dataloader, criterion, optimizer, scaler)
    )
    train_loss, train_acc = result
    # print results
    print('----')
    print(f'batch_size: {batch_size}')
    print(f'image_shape: {size}x{size}')
    print(f'num_workers: {num_workers}')
    print(f'cifar-resnet-{6 * n + 2}')
    print(f'time_cost: {time_cost}')
    print(f'train_loss: {train_loss}, train_acc: {train_acc}')
    print('----')
    # record time
    time_record.append(time_cost)

In [ ]:
time_record = np.array(time_record)

In [ ]:
np.save(f'time_epoch_s{size}_b{begin_size}_e{end_size}_g{step_size}_origin.npy', time_record)